In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Atherosclerosis/GSE123086'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"A validated single-cell-based strategy to identify diagnostic and therapeutic targets in complex diseases [study of 13 diseases]"
!Series_summary	"We conducted prospective clinical studies to validate the importance of CD4+ T cells in 13 diseases from the following ICD-10-CM chapters: Neoplasms (breast cancer, chronic lymphocytic leukemia); endocrine, nutritional and metabolic diseases (type I diabetes, obesity); diseases of the circulatory system (atherosclerosis); diseases of the respiratory system (acute tonsillitis, influenza, seasonal allergic rhinitis, asthma); diseases of the digestive system (Crohn’s disease [CD], ulcerative colitis [UC]); and diseases of the skin and subcutaneous tissue (atopic eczema, psoriatic diseases)."
!Series_summary	"Study participants were recruited by clinical specialists based on diagnostic criteria defined by organizations representing each specialist’s discipline. Age and gender matched healthy controls (n = 1

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Sample Characteristics Dictionary
sample_characteristics = {
    0: ['cell type: CD4+ T cells'],
    1: ['primary diagnosis: ASTHMA', 'primary diagnosis: ATHEROSCLEROSIS', 'primary diagnosis: BREAST_CANCER', 'primary diagnosis: CHRONIC_LYMPHOCYTIC_LEUKEMIA', 'primary diagnosis: CROHN_DISEASE', 'primary diagnosis: ATOPIC_ECZEMA', 'primary diagnosis: HEALTHY_CONTROL', 'primary diagnosis: INFLUENZA', 'primary diagnosis: OBESITY', 'primary diagnosis: PSORIASIS', 'primary diagnosis: SEASONAL_ALLERGIC_RHINITIS', 'primary diagnosis: TYPE_1_DIABETES', 'primary diagnosis: ACUTE_TONSILLITIS', 'primary diagnosis: ULCERATIVE_COLITIS'],
    2: ['Sex: Male', 'diagnosis2: ATOPIC_ECZEMA', 'Sex: Female', 'diagnosis2: ATHEROSCLEROSIS', 'diagnosis2: ASTHMA_OBESITY', 'diagnosis2: ASTHMA', 'diagnosis2: ASTMHA_SEASONAL_ALLERGIC_RHINITIS', 'diagnosis2: OBESITY'],
    3: ['age: 56', 'Sex: Male', 'age: 20', 'age: 51', 'age: 37', 'age: 61', 'age: 31', 'age: 41', 'age: 80', 'age: 53', 'age: 73', 'age: 60', 'age: 76', 'age: 77', 'age: 74', 'age: 69', 'age: 81', 'age: 70', 'age: 82', 'age: 67', 'age: 78', 'age: 72', 'age: 66', 'age: 36', 'age: 45', 'age: 65', 'age: 48', 'age: 50', 'age: 24', 'age: 42'],
    4: [float('nan'), 'age: 63', 'age: 74', 'age: 49', 'age: 60', 'age: 68', 'age: 38', 'age: 16', 'age: 12', 'age: 27']
}

# Check if the dataset contains gene expression data
is_gene_available = True

# Determine the availability of the variables 'Atherosclerosis', 'age', and 'gender'
if 1 in sample_characteristics:
    if 'ATHEROSCLEROSIS' in [item.split(': ')[1] for item in sample_characteristics[1]]:
        trait_row = 1

if 3 in sample_characteristics:
    age_values = [item.split(': ')[1] for item in sample_characteristics[3] if 'age' in item]
    if len(set(age_values)) > 1:
        age_row = 3

if 2 in sample_characteristics:
    gender_values = [item.split(': ')[1] for item in sample_characteristics[2] if 'Sex' in item]
    if len(set(gender_values)) > 1:
        gender_row = 2

# Define conversion functions
def convert_trait(value):
    try:
        value = value.split(': ')[1].lower()
        return 1 if value == 'atherosclerosis' else 0
    except:
        return None

def convert_age(value):
    try:
        return int(value.split(': ')[1])
    except:
        return None

def convert_gender(value):
    try:
        value = value.split(': ')[1].lower()
        return 1 if value == 'male' else 0
    except:
        return None

save_cohort_info('GSE123086', './preprocessed/Atherosclerosis/cohort_info.json', is_gene_available, trait_row is not None)

if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Atherosclerosis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Atherosclerosis/trait_data/GSE123086.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM3494884': [0, 56, 1], 'GSM3494885': [0, None, 0], 'GSM3494886': [0, 20, 0], 'GSM3494887': [0, 51, 0], 'GSM3494888': [0, 37, 1], 'GSM3494889': [0, 61, 1], 'GSM3494890': [0, None, 0], 'GSM3494891': [0, 31, 1], 'GSM3494892': [0, 56, 0], 'GSM3494893': [0, 41, 0], 'GSM3494894': [0, 61, 0], 'GSM3494895': [1, None, 0], 'GSM3494896': [1, 80, 1], 'GSM3494897': [1, 53, 1], 'GSM3494898': [1, 61, 1], 'GSM3494899': [1, 73, 1], 'GSM3494900': [1, 60, 1], 'GSM3494901': [1, 76, 1], 'GSM3494902': [1, 77, 0], 'GSM3494903': [1, 74, 0], 'GSM3494904': [1, 69, 1], 'GSM3494905': [0, 77, 0], 'GSM3494906': [0, 81, 0], 'GSM3494907': [0, 70, 0], 'GSM3494908': [0, 82, 0], 'GSM3494909': [0, 69, 0], 'GSM3494910': [0, 82, 0], 'GSM3494911': [0, 67, 0], 'GSM3494912': [0, 67, 0], 'GSM3494913': [0, 78, 0], 'GSM3494914': [0, 67, 0], 'GSM3494915': [0, 74, 1], 'GSM3494916': [0, None, 0], 'GSM3494917': [0, 51, 1], 'GSM3494918': [0, 72, 1], 'GSM3494919': [0, 66, 1], 'GSM3494920': [0, 80, 0], 'GSM3494921': [0, 36, 1], 'GS

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1', '2', '3', '9', '10', '12', '13', '14', '15', '16', '18', '19',
       '20', '21', '22', '23', '24', '25', '26', '27'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '9', '10'], 'ENTREZ_GENE_ID': ['1', '2', '3', '9', '10'], 'SPOT_ID': [1.0, 2.0, 3.0, 9.0, 10.0]}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Define keys for identifiers and gene symbols based on the preview from STEP5
identifier_key = 'ID'
gene_symbol_key = 'ENTREZ_GENE_ID'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping and get the resulting gene expression dataframe
gene_data = apply_gene_mapping(gene_data, mapping_df)
